In [1]:
# Set local python and nltk paths
import sys
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions')
sys.path
import nltk
nltk.data.path = ['/vol/medic02/users/ag6516/nltk_data']

In [29]:
%matplotlib inline

import os
import csv
import json
import pickle
import pandas as pd

from sklearn import model_selection
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
from random import randint
random.seed(123)
np.random.seed(123)

import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

from collections import Counter

In [3]:
dir = ('/vol/medic02/users/ag6516/x_ray_fracture_localisation/')
# dir = ('/Users/Aydan/PhD/x_ray_fracture_localisation/')
df = pd.read_csv(dir + 'cleaned_reports_edited.csv')

In [4]:
reports = []
reports_tok = []
labels = []
words = []
report_lengths = []
num_sentences = []

for i, row in df.iterrows():
    report = row['Report text'].decode('utf-8').lower()
    num_sentences.append(report.count('.'))
    # Remove markdown
    for char in ['\n', '\b']:
        report = report.replace(char, '')
    
    # Replace unnecessary punctuation 
    for char in ['~', '"']:
        report = report.replace(char, '')
        
    for char in ['!', '?', ';', ':', '.']:
        report = report.replace(char, ' . ')
    
    for char in ['(', ')', ',', '/', '\\']:
        report = report.replace(char, ' , ')
        
    # filter out 'comparison' + dates
    #report = re.sub('( compar.*?\d{4})', '', report)
    report = re.sub('(compar.*?\d{4})', '', report)
    report = re.sub('(compar.*?\d{2})', '', report)
    report = re.sub('(xr knee both)', '', report)
    report = re.sub('(xr knee)', '', report)
    report = re.sub('(previous.*comparison)', '', report)
    report = re.sub('( please.*?\.)', '', report)

    # Tokenize
    report_tok = word_tokenize(report)
    report_length = len(report_tok)
    
    # Remove stopwords
    filtered_report_tok = [word for word in report_tok if word not in stopwords.words('english')]
    filtered_report = ' '.join(filtered_report_tok)
    
    reports.append(filtered_report)
    reports_tok.append(filtered_report_tok)
    labels.append(row['Accession'])
    [words.append(word) for word in filtered_report_tok]
    report_lengths.append(report_length)

In [5]:
print 'Vocab length: ', len(sorted(set(words)))
fdist_all = nltk.FreqDist(words)
freqs = [freq for word, freq in fdist_all.most_common() if word not in ['.', ',']]
print 'Avg no. of appearances: ', np.mean(freqs)
print 'Max: ', fdist_all.most_common(2)
print 'STD: ', np.std(freqs)
#print fdist_all.hapaxes()
print 'Number of reports: ', len(reports)
print 'Average length of report: ', np.mean(report_lengths)
print 'STD: ', np.std(report_lengths)
print ''
print 'Average number of sentences: ', np.mean(num_sentences)
print 'STD: ', np.std(num_sentences)
print ''
more_than_5 = [i for i in freqs if i >= 5]
print 'Percentage of words that appear >=5 times: ', 100*len(more_than_5)//len(freqs)
df['Cleaned report'] = reports
print df.count()

Vocab length:  1735
Avg no. of appearances:  35.4039238315
Max:  [(u'.', 12474), (u'joint', 3749)]
STD:  170.997692276
Number of reports:  3561
Average length of report:  30.5237292895
STD:  18.2751155872

Average number of sentences:  2.70176916596
STD:  1.96696094788

Percentage of words that appear >=5 times:  34
Unnamed: 0          3561
Accession           3561
Clinical history    2221
Comment             2390
Report text         3561
Cleaned report      3561
dtype: int64


In [6]:
patient_ids = []
for folder in os.listdir(dir+'data/Images'):
    #os.path.exists(self.labelfilename)
    patient_ids.append(str(folder))

In [7]:
print len(patient_ids)

330


In [35]:
sample_exams = []
views = {}
APL_views = ['L_R', 'L_L', 'AP_L', 'AP_R', 'HBL_L', 'HBL_R']
tot_APL_views = []

for pid in patient_ids:
    item = {}
    item['patient id'] = pid
    apl_views = 0
    
    if pid in labels:
        item['report'] = reports[labels.index(pid)]
        item['tokens'] = reports_tok[labels.index(pid)]
        images = []
        for image in os.listdir(dir + 'data/Images/' + str(pid)):
            if image.endswith('.jpg'):
                image_dict = {}
                imid = os.path.splitext(image)[0] 
                image_dict['impath'] = pid + '/' + image

                if '_' in image and len(imid.split('_')) > 2:  
                    leg = imid.split('_')[-1]
                    view = imid.split('_')[-2] + '_' + leg

                    image_dict['imid'] = imid.split('_')[0]
                    image_dict['leg'] = leg
                    image_dict['view'] = view
                    if view in APL_views:
                        apl_views = apl_views+1
                    if view not in views.keys():
                        views[view] = 0
                    views[view] = views[view] + 1
                else:
                    apl_views = -1
                    image_dict['imid'] = imid

                images.append(image_dict)
        item['images'] = images
        sample_exams.append(item)
    else: continue     

    tot_APL_views.append(apl_views)

print 'Total exams: ', len(sample_exams)
print ''
print 'Views:'
print views
with_views = [views for views in tot_APL_views if views>=0]
more_than_4 = [views for views in tot_APL_views if views>=4]
print len(with_views), len(more_than_4)

Total exams:  327

Views:
{'L_R': 43, 'HBL_L': 7, 'S_R': 20, 'AP_R': 38, 'WBAP_R': 19, 'WBL_R': 9, 'WBL_L': 8, 'AP_L': 39, 'WBAP_L': 22, 'HBL_R': 5, 'S_L': 20, 'L_L': 47, 'WBAP_B': 1}
63 36


In [36]:
print sample_exams[0]

{'patient id': 'RJ109723379', 'report': u'. images acquired orthopaedic . significant bony abnormality seen .', 'images': [{'imid': 'CR.1.2.840.113564.1018830201.20150216093656437190.1003000225002', 'impath': 'RJ109723379/CR.1.2.840.113564.1018830201.20150216093656437190.1003000225002.jpg'}, {'imid': 'CR.1.2.840.113564.1018830201.20150216093656468220.1003000225002', 'impath': 'RJ109723379/CR.1.2.840.113564.1018830201.20150216093656468220.1003000225002.jpg'}, {'imid': 'CR.1.3.12.2.1107.5.4.4.1302.30000015021410095576500000344', 'impath': 'RJ109723379/CR.1.3.12.2.1107.5.4.4.1302.30000015021410095576500000344.jpg'}, {'imid': 'CR.1.3.12.2.1107.5.4.4.1302.30000015021410095576500000336', 'impath': 'RJ109723379/CR.1.3.12.2.1107.5.4.4.1302.30000015021410095576500000336.jpg'}, {'imid': 'CR.1.3.12.2.1107.5.4.4.1302.30000015021410095576500000330', 'impath': 'RJ109723379/CR.1.3.12.2.1107.5.4.4.1302.30000015021410095576500000330.jpg'}, {'imid': 'CR.1.3.12.2.1107.5.4.4.1302.3000001502141009557650000

## Reduce vocab

In [43]:
allwords = Counter()
pids = set()
for item in sample_exams:
    pid = item['patient id']
    if pid not in pids:
        tokens = item['tokens']
        allwords.update(tokens)

    pids.add(pid) 

vocab = [k for k, v in allwords.items() if v >= 5]

len(vocab)

165

In [44]:
for item in sample_exams:
    old_tokens = item['tokens']
    item['tokens'] = [word for word in old_tokens if word in vocab]

In [49]:
train, test_val = model_selection.train_test_split(sample_exams, train_size=0.8, random_state=42)
test, val = model_selection.train_test_split(test_val, train_size=0.5, random_state=42)

for item in train:
    item['split'] = 'train'

for item in val:
    item['split'] = 'val'

for item in test:
    item['split'] = 'test'

In [50]:
sample_exams = train+test+val
print 'Train: ', len(train)
print 'Val: ', len(val)
print 'Test: ', len(test)
print 'Total', len(sample_exams)

Train:  261
Val:  33
Test:  33
Total 327


In [51]:
pickle.dump(sample_exams, open('sample_exams_train_test_split.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)

In [48]:
for exam in sample_exams:
    print len(exam['images'])

4
4
6
4
4
6
4
6
4
5
4
6
6
4
4
4
1
6
7
4
4
6
4
4
6
4
6
4
4
4
4
4
6
4
4
4
6
6
4
6
4
5
4
8
4
4
4
4
1
4
4
4
6
4
4
4
4
4
4
4
4
4
6
4
4
2
5
6
4
6
4
4
4
4
6
4
4
4
6
4
4
4
6
6
6
6
4
6
4
4
4
4
6
6
6
6
4
4
4
6
4
4
4
6
4
4
4
4
4
4
4
4
4
4
6
4
4
7
4
6
4
5
4
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6
4
5
4
6
4
7
4
6
4
4
4
4
2
4
4
6
6
4
4
6
4
4
4
4
4
4
6
4
4
4
6
6
4
6
4
4
2
4
4
4
6
6
4
6
6
4
6
2
4
3
4
6
4
6
4
5
4
6
4
6
6
4
4
6
7
7
4
6
4
4
1
4
4
6
4
4
6
4
6
4
4
4
6
4
6
4
4
6
6
4
4
4
6
4
4
4
4
6
4
4
4
4
6
4
2
6
6
6
4
4
6
6
4
4
4
4
4
4
6
6
6
4
6
4
4
4
4
2
4
6
4
6
4
5
6
6
4
4
4
4
4
4
4
4
4
6
4
6
4
4
6
6
6
4
4
4
4
2
6
4
4
4
4
4
4
4
6
4
6
4
4
6
4
4
4
4
4
6
6
4
6
